# Spelling Correction using encoder Transformer

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/spelling-correction-encoder-transformer](https://github.com/huseinzol05/Malaya/tree/master/example/spelling-correction-encoder-transformer).
    
</div>

In [1]:
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
import malaya

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
# some text examples copied from Twitter

string1 = 'krajaan patut bagi pencen awal skt kpd warga emas supaya emosi'
string2 = 'Husein ska mkn aym dkat kampng Jawa'
string3 = 'Melayu malas ni narration dia sama je macam men are trash. True to some, false to some.'
string4 = 'Tapi tak pikir ke bahaya perpetuate myths camtu. Nanti kalau ada hiring discrimination despite your good qualifications because of your race tau pulak marah. Your kids will be victims of that too.'
string5 = 'DrM cerita Melayu malas semenjak saya kat University (early 1980s) and now as i am edging towards retirement in 4-5 years time after a career of being an Engineer, Project Manager, General Manager'
string6 = 'blh bntg dlm kls nlp sy, nnti intch'
string7 = 'mulakn slh org boleh ,bila geng tuh kena slhkn jgk xboleh trima .. pelik'

### Load Encoder transformer speller

This spelling correction is a transformer based, improvement version of `malaya.spelling_correction.probability.Probability`. Problem with `malaya.spelling_correction.probability.Probability`, it naively picked highest probability of word based on public sentences (wiki, news and social media) without understand actual context, example,

```python
string = 'krajaan patut bagi pencen awal skt kpd warga emas supaya emosi'
prob_corrector = malaya.spelling_correction.probability.load()
prob_corrector.correct_text(string)
-> 'kerajaan patut bagi pencen awal sakit kepada warga emas supaya emosi'
```

It supposely replaced `skt` with `sikit`, a common word people use in social media to give a little bit of attention to `pencen`. So, to fix that, we can use Transformer model! 

**Right now transformer speller supported `BERT`, `ALBERT` and `ELECTRA` only**.

```python
def encoder(model, sentence_piece: bool = False, **kwargs):
    """
    Load a Transformer Encoder Spell Corrector. Right now only supported BERT, ALBERT and ELECTRA.

    Parameters
    ----------
    sentence_piece: bool, optional (default=False)
        if True, reduce possible augmentation states using sentence piece.

    Returns
    -------
    result: malaya.spelling_correction.transformer.Transformer class
    """
```

In [4]:
model = malaya.transformer.load(model = 'electra')

INFO:malaya_boilerplate.huggingface:downloading frozen huseinzol05/v34-pretrained-model/electra-base.tar.gz
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /Users/huseinzolkepli/Malaya/electra-model/base/electra-base/model.ckpt


In [5]:
transformer_corrector = malaya.spelling_correction.transformer.encoder(model, sentence_piece = True)

INFO:malaya_boilerplate.huggingface:downloading frozen huseinzol05/v27-preprocessing/sp10m.cased.v4.vocab
INFO:malaya_boilerplate.huggingface:downloading frozen huseinzol05/v27-preprocessing/sp10m.cased.v4.model
INFO:malaya_boilerplate.huggingface:downloading frozen huseinzol05/v27-preprocessing/bm_1grams.json


#### To correct a word

```python
def correct(
    self,
    word: str,
    string: List[str],
    index: int = -1,
    lookback: int = 5,
    lookforward: int = 5,
    batch_size: int = 20,
):
    """
    Correct a word within a text, returning the corrected word.

    Parameters
    ----------
    word: str
    string: List[str]
        Tokenized string, `word` must a word inside `string`.
    index: int, optional (default=-1)
        index of word in the string, if -1, will try to use `string.index(word)`.
    lookback: int, optional (default=5)
        N words on the left hand side.
        if put -1, will take all words on the left hand side.
        longer left hand side will take longer to compute.
    lookforward: int, optional (default=5)
        N words on the right hand side.
        if put -1, will take all words on the right hand side.
        longer right hand side will take longer to compute.
    batch_size: int, optional (default=20)
        batch size to insert into model.

    Returns
    -------
    result: str
    """
```

In [6]:
splitted = string1.split()
transformer_corrector.correct('kpd', splitted)

'kepada'

In [7]:
transformer_corrector.correct('krajaan', splitted)

'kerajaan'

In [8]:
%%time

transformer_corrector.correct('skt', splitted)

CPU times: user 19.2 s, sys: 837 ms, total: 20 s
Wall time: 3.59 s


'sikit'

In [9]:
%%time

transformer_corrector.correct('skt', splitted, lookback = -1)

CPU times: user 18.8 s, sys: 917 ms, total: 19.7 s
Wall time: 3.78 s


'sikit'

In [10]:
%%time

transformer_corrector.correct('skt', splitted, lookback = 2)

CPU times: user 12.6 s, sys: 588 ms, total: 13.2 s
Wall time: 2.37 s


'sikit'

#### To correct a sentence

```python
def correct_text(
    self,
    text: str,
    lookback: int = 5,
    lookforward: int = 5,
    batch_size: int = 20
):
    """
    Correct all the words within a text, returning the corrected text.

    Parameters
    ----------
    text: str
    lookback: int, optional (default=5)
        N words on the left hand side.
        if put -1, will take all words on the left hand side.
        longer left hand side will take longer to compute.
    lookforward: int, optional (default=5)
        N words on the right hand side.
        if put -1, will take all words on the right hand side.
        longer right hand side will take longer to compute.
    batch_size: int, optional(default=20)
        batch size to insert into model.

    Returns
    -------
    result: str
    """
```

In [11]:
transformer_corrector.correct_text(string1)

'kerajaan patut bagi pencen awal sikit kepada warga emas supaya emosi'

In [12]:
tokenizer = malaya.tokenizer.Tokenizer()

In [18]:
string2

'Husein ska mkn aym dkat kampng Jawa'

In [16]:
tokenized = tokenizer.tokenize(string2)
transformer_corrector.correct_text(' '.join(tokenized))

'Husein suka mkn ayam dikota kampung Jawa'

In [17]:
tokenized = tokenizer.tokenize(string3)
transformer_corrector.correct_text(' '.join(tokenized))

'Melayu malas ini narration dia sama sahaja macam men are trash . True to some , false to some .'

In [13]:
tokenized = tokenizer.tokenize(string5)
transformer_corrector.correct_text(' '.join(tokenized))

'DrM cerita Melayu malas semenjak saya kat University ( early 1980s ) and now as i am edging towards retirement in 4 - 5 years time after a career of being an Engineer , Project Manager , General Manager'

In [14]:
tokenized = tokenizer.tokenize(string6)
transformer_corrector.correct_text(' '.join(tokenized))

'boleh buntong dalam kelas nlp saye , nanti intch'

In [15]:
tokenized = tokenizer.tokenize(string7)
transformer_corrector.correct_text(' '.join(tokenized))

'mulakan salah orang boleh , bila geng itu kena salahkan juga xboleh terima . . pelik'